# 2. Les stations les plus proches avec géopandas

On se sert de géopandas et des geodataframes vus précedemment pour trouver les stations les plus proche de la position de l'usager.

In [5]:
import pandas as pd 
import geopandas as gpd
import urllib
import shapely
pd.options.mode.chained_assignment = None  # default='warn'
from math import *
from geopy.geocoders import Nominatim
import module_projet as md

On prend donc le geodataframe avec données instantannées et on le nettoie en enlevant les doublons et en gardant les colonnes qui nous intéressent. 
Ensuite, on vérifie dans quel système de projection est le geodataframe.

In [3]:
dat_inst_g = gpd.read_file('data/fichier_inst_g.geojson')

In [6]:
data_g = md.data_carb(dat_inst_g, 'Gazole',['adresse', 'ville', 'prix_valeur', 'prix_nom', 'geometry'])

In [20]:
data_g.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

On fait maintenant une fonction pour trouver les stations les 10 stations les plus proches en faisant attention de se projeter en projection Lambert pour faire le calcul de distance puisque c'est le système de projection officiel en France. 

In [24]:
def dix_plus_proche_g(df, posi, carb): 
    data = md.data_carb(df, carb,['adresse', 'ville', 'prix_valeur', 'prix_nom', 'geometry'])
    p = shapely.geometry.Point(list(map(float, md.Adresse_to_gps(posi).split(','))))
    col_ad = gpd.GeoSeries((p for i in range(len(data))),index= data.index, crs = 4326)
    new_data = data.to_crs(2154)
    new_col = col_ad.to_crs(2154)
    data_1 = new_data.distance(new_col).sort_values().head(10)
    return data.loc[data_1.index.to_list()]

In [25]:
dix_plus_proche_g(data_g, "1 rue de Rivoli, Paris, France", "Gazole")

,adresse,ville,prix_valeur,prix_nom,geometry
6321,Cala Longa,Bonifacio,1.768,Gazole,POINT (9.16600 41.39200)
2200,sennola,BONIFACIO,1.767,Gazole,POINT (9.16500 41.39100)
10989,358 San Cipriano,Lecci,1.750,Gazole,POINT (9.33600 41.63900)
28068,Route de Bonifacio,Porto-Vecchio,1.800,Gazole,POINT (9.27300 41.57100)
17308,Route de bonifacio BP115,Porto-Vecchio,1.750,Gazole,POINT (9.27469 41.57844)
11092,Les 4 Chemins BP115,Porto-Vecchio,1.750,Gazole,POINT (9.27600 41.59800)
19655,Station TOTAL - Quartier La Poretta,PORTO-VECCHIO,1.740,Gazole,POINT (9.27600 41.60200)
29251,RTE N 198,Porto-Vecchio,1.750,Gazole,POINT (9.27700 41.60400)
31149,route de bastia,Porto-Vecchio,1.740,Gazole,POINT (9.28000 41.61400)
13066,Station TOTAL La Trinité - RN 198 - Pitrera,PORTO-VECCHIO,1.775,Gazole,POINT (9.29420 41.63490)


Le résultat n'est pas concluant comme on peut rapidement le voir et malgré de nombreuses recherches et une attention particulière portée sur les systèmes de projection, nous n'avons pas réussi à trouver pourquoi. C'est pourquoi nous allons par la suite résonner avec des dataframes classiques.